Zixuan Chen 61665307

## Setup

In [1]:
import numpy as np
from time import perf_counter as clock
from beautifultable import BeautifulTable
np.random.seed(0)
from hw4_helper import *

### Content of hw4_helper
My Funtion Object implemented in Homework2 and Homework3. The differetiation method is implemented using Sympy library.\
My Conjugate Gradient method implemented in Homework2 using backtracking line search. It will be used in Problem 1.\
$$-Source:Kochenderfer \& Wheeler book, Algorithm 4.2 (pg.56), "backtracking\_line\_search".$$$$ Equation 5.16 (pg.73), "Fletcher-Reeves". Algorithm 5.2 (pg.74), "ConjugateGradientDescent".$$
Helper Functions:
$$array(*args, ndmin = 1) -> np.array$$
$$gradient(f:Function) -> array(Function)$$
$$compute\_vector(v: [Function], x: np.array, ndmin = 1) -> np.array$$
$$gradient\_magnitude(g: [Function], x: np.array) -> float$$
$$vector\_length(v) -> float$$

# Problem 1  Constrained Optimization
## Augmented Lagrange Method
The method I will use for this problem is the augmented Lagrangian method.
$$-Source: Kochenderfer and Wheeler book (pg.183) Algorithm 10.2, "augmented\_lagrange\_method".$$
"penalty_function" will turn the penalty function into a Function object, then I can use its methods to take the gradient.\
The termination condition for Augmented Lagrange Method is 
$$||h(x)|| <= \epsilon$$
However, there are chances that the start pointing is carefully designed to make h(x) = 0 before reaching the optimum. To escape the carefully designed point, I add an additional condition that it will run for at least 5 iterations.

In [2]:
def penalty_function(f, rho, h, x, lmda) -> (Function, [Function]):
    '''x -> f(x) + rho / 2 * sum(compute_vector(h, x)**2) - lmda.dot(compute_vector(h, x))'''
    if len(h) == 0:
        return f, gradient(f)
    f_str = f.func + '+' + str(rho/2) + '*((' 
    f_str += '+('.join([h_i.func +')**2' for h_i in h]) + ')-'
    f_str += '-'.join([f'{lmda[i]}*({h[i]})' for i in range(len(h))])
    p = Function(f_str, ','.join([f'x{i}' for i in range(len(x))]))
    return p, gradient(p)            
    
def augmented_lagrange(f, f_prime, h, initial_x, rho = 1, gama = 2, tol = 1e-4):
    start_time = clock()
    x = initial_x.copy()
    hx = compute_vector(h, x) # h(x)
    lmda = np.zeros(len(hx))
    iteration = 0
    while vector_length(hx) > tol or iteration < 5:
        iteration += 1
        penalty, penalty_prime = penalty_function(f, rho, h, x, lmda)
        x = conjugate_gradients(penalty, x, penalty_prime) #; print(x)
        hx = compute_vector(h, x)
        rho *= gama
        lmda -= rho * hx
    return x, vector_length(hx), f(x), iteration, clock() - start_time

## Test & Report
Here I define two test functions:
$$test\_function1 = \sum_{i=0}^{9} (x_{i}-1)^2$$
$$test\_function2 = \sum_{i=0}^{8} (1-x_{i})^2 + 5(x_{i+1}-x_{i}^2)^2$$
And two constraints:
$$(inactive)h1 = \sum_{i=0}^{9} x_{i} = 0 $$
$$(active)h2 = x_0 - 2 = 0$$

In [3]:
# 10-dim (x-1)^2
f_str = ' + '.join([f'(x{i} - 1)**2' for i in range(10)])
test_func1 = Function(f_str, ','.join([f'x{i}' for i in range(10)]), name = 'Quadratic')

# 10-dim Rosenbrock 
f_str = ' + '.join([f'(1 - x{i})**2 + 5*(x{i+1} - x{i}**2)**2' for i in range(9)])
test_func2 = Function(f_str, ','.join([f'x{i}' for i in range(10)]), name = 'Rosenbrock')

# Equality Constraints
h1 = ' + '.join([f'x{i}' for i in range(10)]) + ' - 10'
h1 = Function(h1, ','.join([f'x{i}' for i in range(10)]))
h2 = Function('x0 - 2', 'x0')

for i, tf in enumerate([test_func1, test_func2], 1):
    print(f'Test Function {i}: {tf.name}\n{tf}\nvariables: {tf.symbols}\n')
for i, c in enumerate([h1, h2], 1):
    print(f'Equality Constraint {i}: {c.name}\n{c} = 0\nvariables: {c.symbols}\n')

Test Function 1: Quadratic
(x0 - 1)**2 + (x1 - 1)**2 + (x2 - 1)**2 + (x3 - 1)**2 + (x4 - 1)**2 + (x5 - 1)**2 + (x6 - 1)**2 + (x7 - 1)**2 + (x8 - 1)**2 + (x9 - 1)**2
variables: ['x0', 'x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9']

Test Function 2: Rosenbrock
(1 - x0)**2 + 5*(x1 - x0**2)**2 + (1 - x1)**2 + 5*(x2 - x1**2)**2 + (1 - x2)**2 + 5*(x3 - x2**2)**2 + (1 - x3)**2 + 5*(x4 - x3**2)**2 + (1 - x4)**2 + 5*(x5 - x4**2)**2 + (1 - x5)**2 + 5*(x6 - x5**2)**2 + (1 - x6)**2 + 5*(x7 - x6**2)**2 + (1 - x7)**2 + 5*(x8 - x7**2)**2 + (1 - x8)**2 + 5*(x9 - x8**2)**2
variables: ['x0', 'x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9']

Equality Constraint 1: 
x0 + x1 + x2 + x3 + x4 + x5 + x6 + x7 + x8 + x9 - 10 = 0
variables: ['x0', 'x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9']

Equality Constraint 2: 
x0 - 2 = 0
variables: ['x0']



In [4]:
def sample_solution(solutions):
    return set(', '.join(map(lambda x: str(round(x, 3)), s)) for s in solutions)
    
def test_problem1(test_func, func_prime, constraints, initial_points, sample_size):
    result = np.array([augmented_lagrange(test_func, func_prime, constraints, initial_points[i]) for i in range(sample_size)])
    solutions, result = result[:,0], result[:,1:]
    
    table = BeautifulTable()
    table.set_style(BeautifulTable.STYLE_COMPACT)
    header = test_func.name+' s.t.\n'+'\n'.join([str(c)+' = 0' for c in constraints])
    table.column_headers = [header]
    
    report = BeautifulTable()
    table.append_row([report])
    report.column_headers = ["||h(x)||", "f(x)", "Iteration", "time"]
    
    row = []
    for i in range(result.shape[1]):
        m = np.mean(result[:,i])  # sample mean
        s = np.sqrt(sum([(k - m)**2 for k in result[:,i]]) / (sample_size - 1)) # unbiased estimate of population std
        row.append('mean:{:.3e}\nstd:{:.3e}'.format(m,s))
    report.append_row(row)
    return sample_solution(solutions), table

The method will be tested on 20 randomly selected initial points. Each is a 10-dimensional point in [-20, 20].\
There will also be two seperated tests. One uses only the inactive constraint, the other use both constraints. The first test result should look similar to reports from previous homeworks.\
Under the table, I also provide a sample solution, a non-repetitive set of solution given by the runs (rounded to three decimal places).\
For the report, there are four things I interested in:\
(1) ||h(x)||: How well the solution satisfied the constraints. This is expected to be very small.\
(2) f(x): Function values. Two test functions are designed to have a global mimunm at f(x) = 0. Therefore the closer to zero, the better the solution is.\
(3) Iteration and (4) Running time: Both indicate how efficient the method is.

In [5]:
sample_size = 20
N = np.random.uniform(-20, 20, (sample_size, 10))
for func in [test_func1, test_func2]:
    solution, table = test_problem1(func, gradient(func), array(h1), N, sample_size)
    print(table)
    print('Sample Solution: ', *solution)
    print() # empty line

                            Quadratic s.t.                             
       x0 + x1 + x2 + x3 + x4 + x5 + x6 + x7 + x8 + x9 - 10 = 0        
-----------------------------------------------------------------------
 +----------------+----------------+----------------+----------------+ 
 |    ||h(x)||    |      f(x)      |   Iteration    |      time      | 
 +----------------+----------------+----------------+----------------+ 
 | mean:1.416e-06 | mean:1.327e-12 | mean:5.000e+00 | mean:3.468e-01 | 
 | std:8.169e-07  | std:2.817e-12  | std:0.000e+00  | std:4.651e-02  | 
 +----------------+----------------+----------------+----------------+ 
Sample Solution:  1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0

                            Rosenbrock s.t.                            
       x0 + x1 + x2 + x3 + x4 + x5 + x6 + x7 + x8 + x9 - 10 = 0        
-----------------------------------------------------------------------
 +----------------+----------------+----------------+---------------

We get the exact same solution and function values as like we are doing unconstrained optimization because the optimal point for both functions are happened to be on the constraint, therefore it didn't stop them converging to the minimum. 

In [6]:
for func in [test_func1, test_func2]:
    solution, table = test_problem1(func, gradient(func), array(h1, h2), N, sample_size)
    print(table)
    print('Sample Solution: ', *solution)
    print() # empty line

                            Quadratic s.t.                             
       x0 + x1 + x2 + x3 + x4 + x5 + x6 + x7 + x8 + x9 - 10 = 0        
                              x0 - 2 = 0                               
-----------------------------------------------------------------------
 +----------------+----------------+----------------+----------------+ 
 |    ||h(x)||    |      f(x)      |   Iteration    |      time      | 
 +----------------+----------------+----------------+----------------+ 
 | mean:8.446e-05 | mean:1.111e+00 | mean:8.000e+00 | mean:6.220e-01 | 
 | std:3.047e-07  | std:8.480e-07  | std:0.000e+00  | std:2.576e-02  | 
 +----------------+----------------+----------------+----------------+ 
Sample Solution:  2.0, 0.889, 0.889, 0.889, 0.889, 0.889, 0.889, 0.889, 0.889, 0.889

                            Rosenbrock s.t.                            
       x0 + x1 + x2 + x3 + x4 + x5 + x6 + x7 + x8 + x9 - 10 = 0        
                              x0 - 2 = 0          

This time the second constraint bounds the first variable to be 2. Thus the solutions are different, and it takes the method more iterations.

# Problem 2  Stochastic Direct Search
## Simulated Annealing
$$-Source:Kochenderfer \& Wheeler book, Algorithm 8.4 (pg.130), "simulated\_annealing".$$

In [7]:
def simulated_annealing(f, initial_x, T, p = 0.95, kmax = 1e4, tol = 1e-10):
    start_time = clock()
    x = initial_x.copy()
    y = f(x)
    x_best, y_best = x, y
    iteration, accept, update = 0, 0, 0
    while iteration < kmax:
        iteration += 1
        # update the point by random amount. Also scale this step based on current temperature
        x_new = x + np.array([np.random.normal() for _ in range(len(x))]) * max(np.sqrt(T), 0.1)
        y_new = f(x_new)
        delta_y = y_new - y
        # Accept the step if it is a downhill change, or the temperature is still high
        if delta_y <= 0 or np.random.normal() < np.exp(-delta_y/T):
            x, y = x_new, y_new
            accept += 1
        if y_new < y_best:
            x_best, y_best = x_new, y_new ;#print(x_best, y_best)
            update += 1
        if T > tol: # stop cooling if it is cold enough
            T *= p    
    return accept/iteration, update/iteration, abs(y_best), clock() - start_time

## Test & Report
The test functions are the same from Problem 1.

In [8]:
def test_problem2(test_func, initial_points, T, sample_size):
    result = np.array([simulated_annealing(test_func, initial_points[i], T) for i in range(sample_size)])
    
    table = BeautifulTable()
    table.set_style(BeautifulTable.STYLE_COMPACT)
    table.column_headers = [test_func.name]
    
    report = BeautifulTable()
    table.append_row([report])
    report.column_headers = ["Acceptance Rate", "Update Rate", "|Error|\n|f(x)|", "time"]
    
    row = []
    for i in range(result.shape[1]):
        m = np.mean(result[:,i])  # sample mean
        s = np.sqrt(sum([(k - m)**2 for k in result[:,i]]) / (sample_size - 1)) # unbiased estimate of population std
        row.append('mean:{:.3e}\nstd:{:.3e}'.format(m,s))
    report.append_row(row)
    return table

The method will be tested on 20 randomly selected initial points. Each is a 10-dimensional point in [-20, 20].\
For the report, as usual, there are four things I interested in:\
(1) Acceptance Rate: How many steps are accepted over number of iteration.\
(2) Update Rate: How many steps actually make progress to the solution.\
(3) |f(x)|: Because test functions are deigned to have global minimum at f(x) = 0, this evaluate how well the method performs. \
(4) Running time: How efficient is this method.

In [9]:
sample_size = 20
T = 10
N = np.random.uniform(-20, 20, (sample_size, 10))
for func in [test_func1, test_func2]:
    print(test_problem2(func, N, T, sample_size))
    print() # empty line

                               Quadratic                                
------------------------------------------------------------------------
 +-----------------+----------------+----------------+----------------+ 
 | Acceptance Rate |  Update Rate   |    |Error|     |      time      | 
 |                 |                |     |f(x)|     |                | 
 +-----------------+----------------+----------------+----------------+ 
 | mean:7.021e-01  | mean:3.837e-02 | mean:2.068e-01 | mean:3.119e-01 | 
 |  std:5.371e-03  | std:8.883e-03  | std:6.911e-02  | std:7.090e-03  | 
 +-----------------+----------------+----------------+----------------+ 

                               Rosenbrock                               
------------------------------------------------------------------------
 +-----------------+----------------+----------------+----------------+ 
 | Acceptance Rate |  Update Rate   |    |Error|     |      time      | 
 |                 |                |     |f(x)|  

Solutions to the Quadratic problem are acceptable. However, the errors of Rosenbrock are large.\
The reason behind this can be explained by the small Update Rate. As the table suggested, a lot of steps are accepted, however, few make actual progress. It is probably too difficult to get the right descent direction in high dimensions.\
It may have better performance if it can have a larger maximum iteration number or a better cooling schedule.